In [1]:
# Iremos criar um agente para interagir com o ambiente escolhendo as melhores ações para cada estado. Esse agente será uma rede
# neural que recebe como entrada o estado e retorna como saída a melhor ação a ser tomada. 
# Em vez de treinar essa rede usando gradiente descendente como estamos acostumados (nesse caso não temos os dados de treinamento
# do que é certo ou errado, e também não temos um algoritmo de RL para guiar o treino da rede neural), iremos iniciar várias 
# redes (várias populações) com pesos aleatórios e iremos avançar usando um algoritmo genético. Ou seja, as redes interagem com 
# o ambiente e recebem um feedback (Fitness Function, que é uma função da recompensa do sistema: pode ser a recompensa total de 
# um episódio, por exemplo). Depois dessa interação, selecionamos as melhores populações de pesos, fazemos crossover + mutação, 
# criamos novas populações de pesos e interagimos essas novas redes com o ambiente para a próxima seleção. No final, escolheremos
# a rede neural mais bem calibrada.

import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import matplotlib.pyplot as plt
import gym
import keras
from keras.models import Sequential
from keras.layers import Dense
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import deap

Using TensorFlow backend.


In [2]:
def organiza_pesos(pesos, dimensoes_rede):

    pesos_para_keras=[]
    desloca=0
    
    for camada in range(len(dimensoes_rede)-1):
        weights=[]
        for neuronio in range(dimensoes_rede[camada]): 
            weights.append(pesos[desloca:desloca+dimensoes_rede[camada+1]]) 
            desloca=desloca+dimensoes_rede[camada+1]
            
        pesos_para_keras.append(np.array(weights)) 
        pesos_para_keras.append(np.array(pesos[desloca:desloca+dimensoes_rede[camada+1]]))
        desloca=desloca+dimensoes_rede[camada+1]

    return pesos_para_keras

In [3]:
# Fitness Function (irá interagir com o ambiente a partir dos dados da rede neural fornecida e retornar a recompensa do episódio)
def cart_pole(individuo):
    maximo_iteracoes = 200
    recompensa_total=0
    env = gym.make('CartPole-v0')
    obs = env.reset()
    modelo.set_weights(organiza_pesos(individuo,[4,10,5,1]))

    for i in range(maximo_iteracoes): # avalia a recompensa total obtida
        acao = modelo.predict_classes(np.array([obs]))[0][0]
        obs, recompensa, terminou, info = env.step(acao)
        recompensa_total+=recompensa
        if terminou or i==maximo_iteracoes-1: # se morreu ou chegou em 200 iterações
            return recompensa_total,
            break

In [4]:
# Criando o modelo da rede neural (sem informar compilador ou algoritmo otimizador, apenas o esqueleto da rede):
# Essa rede terá 4 neurônios de entrada (estado atual), 10 neurônios na 1ª camada oculta, 5 neurônios na segunda e 1 neurônio
# na saída: 
modelo = Sequential()
modelo.add(Dense(10, input_dim=4, activation='relu'))
modelo.add(Dense(5, activation='relu'))
modelo.add(Dense(1, activation='sigmoid')) # A ação do agente no Cartpole consiste em 0 ou 1

# Criando o algoritmo genético:
creator.create(name="ClasseFitness", base=base.Fitness, weights=(1.0,))     
creator.create(name="ClasseIndividuos", base=list, fitness=creator.ClasseFitness) 

# Toolbox
modulo_toolbox = base.Toolbox()
modulo_toolbox.register("pesos_e_bias", random.uniform, -1, 1) # definindo pesos no range [-1, 1]                  
modulo_toolbox.register("individuos", tools.initRepeat, creator.ClasseIndividuos, modulo_toolbox.pesos_e_bias, 111)
# Aqui temos 111 variáveis por indivíduo, pois 4x10 + 10 + 10x5 + 5 + 5x1 + 1 = 111 (qtde de pesos e bias totais da rede)
modulo_toolbox.register("populacao", tools.initRepeat, list, modulo_toolbox.individuos, 100)  # criando uma população de 100 indivíduos
modulo_toolbox.register("evaluate", cart_pole) # informando a Fitness Function
modulo_toolbox.register("mate", tools.cxBlend, alpha=0.5) # informando o crossover
modulo_toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.2, indpb=0.05) # informando a  mutação
modulo_toolbox.register("select", tools.selTournament, tournsize=3) # informando a seleção natural

In [5]:
# Executando o código completo em ordem:
    
populacao = modulo_toolbox.populacao() 
hof = tools.HallOfFame(1) # salvando o melhor indivíduo de todos
estatisticas = tools.Statistics(lambda ind: ind.fitness.values) # criando estatísticas de cada população

estatisticas.register("media", np.mean)
estatisticas.register("dp", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

# Rodando o algoritmo completo:
populacao, log = algorithms.eaSimple(populacao, modulo_toolbox, cxpb=0.8, mutpb=0.2, ngen=10, stats=estatisticas, halloffame=hof, verbose=True)


gen	nevals	media	dp    	min	max
0  	100   	14.32	14.578	8  	98 
1  	92    	13.76	16.2869	8  	134
2  	86    	17.65	23.9747	8  	144
3  	86    	20.17	26.4927	8  	134
4  	80    	19.9 	23.0367	8  	134
5  	86    	20.4 	23.5792	8  	134
6  	78    	25.27	29.4601	8  	200
7  	78    	29.9 	41.1926	8  	200
8  	86    	27.81	36.2755	8  	200
9  	89    	25.45	37.8202	8  	200
10 	85    	31.98	46.3812	8  	200


In [6]:
pesos_finais = hof[0]

In [7]:
pesos_finais

[0.23037917714079267,
 0.2871014439779445,
 -0.1992403367663325,
 -0.21124348542069285,
 -0.08068714114572317,
 -0.4564034946573546,
 -0.05172854304103963,
 -1.222808623820599,
 0.39669633525825065,
 -0.3779192003634627,
 0.29907703377004513,
 0.4651202658280819,
 0.5968757865567011,
 -0.5422285429239698,
 1.0412836785959545,
 0.291456978257598,
 -0.3607589041265303,
 -0.36594210166904756,
 -1.0088851091115902,
 -0.8891654190443707,
 -0.9365481859299615,
 0.33226135271390644,
 -0.0860875044685532,
 -0.09445624140748932,
 -0.5427715215439503,
 0.5683723151883129,
 -0.7560902879340269,
 -0.036927674495694784,
 0.3996893067483838,
 -0.5949634662517709,
 -0.003961394203597451,
 -0.7922626281870047,
 0.20502801172429422,
 -0.12408190703002595,
 -0.20864468128751917,
 0.8048591411853762,
 -0.7056662411332916,
 0.8383779647883376,
 -0.261170725144938,
 0.2865642893420498,
 0.46483248143541483,
 0.14292735884094349,
 1.2543268204149172,
 -1.8434743876197985,
 0.3490904713997146,
 1.10637432812

In [8]:
modelo.set_weights(organiza_pesos(pesos_finais,[4,10,5,1]))

In [9]:
import time
env = gym.make('CartPole-v0')
obs = env.reset()
recompensa_total=0
for i in range(200): 
    env.render()
    time.sleep(0.1)
    acao = modelo.predict_classes(np.array([obs]))[0][0]
    obs, recompensa, terminou, info = env.step(acao)
    recompensa_total+=recompensa
    if terminou:
        break
env.close()
print('Recompensa total:', recompensa_total)

Recompensa total: 200.0


In [10]:
modelo.predict_classes(np.array([obs]))[0][0]

0